### `Input`

In [8]:
# positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
#                       [430, 100], [520, 100], [520, 150],[600, 150], [600, 200], [520, 200]
#                       , [520, 440], [650, 440], [650, 630],
#                       [210, 630], [210, 530], [70, 530], [70, 420],
#                         [150, 420], [150, 140], [50, 140], [50, 40]]
# [ngang, dọc]
grid_size = (60, 100)
positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
                      [430, 100], [520, 100], [520, 150],[600, 150], [600, 220], [620, 220],
                      [620, 180], [660, 180], [660, 280], 
                      [550,280], [550,220], [520, 220], 
                      [520, 440], [650, 440], [650, 630],
                      [210, 630], [210, 530], [70, 530], [70, 420], 
                      [150, 420], [150, 140], [50, 140], [50, 40]]
# positions = np.array([[60, 60], [270, 60], [270, 270], [60, 270], [60, 60]])
# grid_size = [80, 80]
# positions = [[50, 40], [150, 40], [150, 120], [350, 120], 
#              [350, 60], [400, 60], [400, 200], [500, 200], 
#              [500, 500], [50, 500], [50, 40]]



### `TẠO ĐƯỜNG BAO`

In [9]:
from Models import BorderPoint

border_point = BorderPoint(positions)
border_line = border_point.create_border_lines()

## `TẠO MA TRẬN (1->Cạnh, 0->thịt, None->Vùng ngoài)`

In [10]:
from Models import Matrix

mt = Matrix()
# khởi tạo ma trận gồm thịt cạnh và vùng ngoài
mt.create_matrix(grid_size, positions)

# lấy tất cả các đỉnh trong ma trận
corners = mt.take_all_corners()
# thêm các đỉnh vào ma trận
mt.add_corners_to_matrix(corners)
# kết quả
matrix = mt.values()

#### `Check matrix`

In [11]:
# import numpy as np
# import cv2

# shape_res = np.array(matrix).shape
# print(shape_res)

# image = np.ones((700, 700, 3), np.uint8) * 255

# for i in range(0, shape_res[0]):
#     for j in range(0, shape_res[1]):
#         cell = matrix[i][j]
#         x,y,val,gr = cell
#         if cell[2] == 0:
#             cv2.line(image, (x,y), (x,y), (150,200,200),1)
#         elif cell[2] == 1:
#             cv2.line(image, (x,y), (x,y), (0,0,0),1)
        
# for i in range(0, shape_res[0]):
#     for j in range(0, shape_res[1]):
#         x,y,val,gr = matrix[i][j]
#         if val == 2:
#             cv2.line(image, (x,y), (x,y), (50,50,250),4)
            
# trung_diem = (int(np.mean(np.array(border_line)[:,0])), int(np.mean(np.array(border_line)[:,1])))
# x,y = trung_diem
# cv2.line(image, (x,y), (x,y), (0,0,0),5)

# while True:
#     cv2.imshow('DatGach', image)
#     if cv2.waitKey(20) & 0xFF == 27:
#         break

# cv2.destroyAllWindows()

##### `Các mảnh khu 1 - TRÁI TRÊN`

In [12]:
from Models import Core

core = Core()
core.create_core(matrix)
core90, core180, core360 = core.values()

# PHÂN LOẠI VÀ ĐẾM SỐ CÁC HÌNH GIỐNG NHAU

### `TRỰC QUAN HÓA KẾT QUẢ TỔNG KHUNG HÌNH`

In [13]:
import numpy as np
import cv2

image = np.ones((700, 700, 3), np.uint8) * 255
shape_res = np.array(matrix).shape

for i in range(0, shape_res[0]):
    for j in range(0, shape_res[1]):
        cell = matrix[i][j]
        x,y,val,gr = cell
        if cell[2] == 1: # in cạnh
            cv2.line(image, (x,y), (x,y), (0,0,0),1)

[cv2.line(image, (x,y), (x,y), (0,0,0), 5) for [x,y] in core90]
[cv2.line(image, (x,y), (x,y), (0,123,123), 5) for [x,y] in core180]
[cv2.line(image, (x,y), (x,y), (0,0,245), 5) for [x,y] in core360]

while True:
    cv2.imshow('Brick', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()